In [41]:
from selenium import webdriver
from selenium.webdriver.common.by import By # Importar By para localizar elementos
import time # Para adicionar pausas, se necessário
from selenium.webdriver.support.ui import Select
from bs4 import BeautifulSoup
import sys
from collections import defaultdict
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [50]:
class UnidadeUSP:
    def __init__(self, nome):
        self.nome = nome
        self.cursos = []

    def __str__(self):
        return f"Unidade: {self.nome, self.cursos}"
    
    @staticmethod
    def carregar_unidades(driver, limite=None):
        unidades = []
        combo_unidade = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "comboUnidade"))
        )
        WebDriverWait(driver, 10).until(lambda d: len(Select(combo_unidade).options) > 1)
        select_unidade = Select(combo_unidade)
        #print(select_unidade.options)
        total_opcoes = len(select_unidade.options)
        if total_opcoes < 2:  # Deve ter pelo menos 1 opção além do vazio
            raise ValueError("Nenhuma unidade carregada no comboUnidade.")
    
        # Limitar ao menor valor entre o número de opções e o limite especificado
        limite = min(limite, total_opcoes - 1)  
       
        # Carregar as unidades
        for i in range(1, 4):  # Ignorar a primeira opção vazia
            nome_unidade = select_unidade.options[i].text.strip()
            if nome_unidade:  # Ignorar opções vazias
                unidade = UnidadeUSP(nome_unidade)

                # Selecionar a unidade e carregar os cursos
                select_unidade.select_by_index(i)
                unidade.cursos = CursoUSP.carregar_cursos(driver)

                unidades.append(unidade)
        return unidades

    def carregar_cursos(self, driver):
        combo_unidade = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "comboUnidade"))
        )
        select_unidade = Select(combo_unidade)
        
        # Selecionar a unidade
        for option in select_unidade.options:
            if self.nome in option.text:
                select_unidade.select_by_visible_text(option.text)
                break
        
        # Esperar os cursos serem carregados
        WebDriverWait(driver, 10).until(
            lambda d: len(Select(d.find_element(By.ID, "comboCurso")).options) > 1
        )
        
        # Carregar cursos
        self.cursos = CursoUSP.carregar_cursos(driver)
        

In [51]:
class CursoUSP:
    @staticmethod
    def carregar_cursos(driver):
        combo_curso = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "comboCurso"))
        )
        select_curso = Select(combo_curso)
        
        cursos = []
        for option in select_curso.options:
            nome_curso = option.text.strip()
            if nome_curso:  # Ignorar opções vazias
                cursos.append(nome_curso)
        return cursos

In [52]:
def main():
    driver = webdriver.Chrome()
    driver.get("https://uspdigital.usp.br/jupiterweb/jupCarreira.jsp?codmnu=8275")

    unidades = UnidadeUSP.carregar_unidades(driver, limite=3)
    
    for unidade in unidades:
        print(f"Carregando cursos para a unidade: {unidade.nome}")
        unidade.carregar_cursos(driver)
        print(unidade)  # Exibir os cursos associados à unidade
        
        # Associar cursos à unidade atual

    # Fechar o navegador ao terminar
    driver.quit()

if __name__ == "__main__":
    main()

Carregando cursos para a unidade: Escola de Artes, Ciências e Humanidades - ( EACH )
Unidade: ('Escola de Artes, Ciências e Humanidades - ( EACH )', ['Bacharelado em Biotecnologia (Ciclo Básico) - integral', 'Bacharelado em Educação Física e Saúde (Ciclo Básico) - integral', 'Bacharelado em Gestão Ambiental (Ciclo Básico) - matutino', 'Bacharelado em Gestão Ambiental (Ciclo Básico) - noturno', 'Bacharelado em Lazer e Turismo (Ciclo Básico) - vespertino', 'Bacharelado em Lazer e Turismo (Ciclo Básico) - noturno', 'Bacharelado em Sistemas de Informação (Ciclo Básico) - matutino', 'Bacharelado em Sistemas de Informação (Ciclo Básico) - noturno', 'Bacharelado em Têxtil e Moda (Ciclo Básico) - matutino', 'Gerontologia (Ciclo Básico) - vespertino', 'Gestão de Políticas Públicas (Ciclo Básico) - matutino', 'Gestão de Políticas Públicas (Ciclo Básico) - noturno', 'Licenciatura em Ciências da Natureza (Ciclo Básico) - noturno', 'Marketing (Ciclo Básico) - matutino', 'Marketing (Ciclo Básico) - 